In [3]:
import pandas as pd

train_df = pd.read_csv("./data/rainfall_train.csv")
test_df = pd.read_csv("./data/rainfall_test.csv")

train_df = train_df.drop(columns=['Unnamed: 0', 'rainfall_train.fc_year', 'rainfall_train.fc_month',
       'rainfall_train.fc_day', 'rainfall_train.fc_hour'])
test_df = test_df.drop(columns=['Unnamed: 0', 'rainfall_test.fc_year', 'rainfall_test.fc_month',
       'rainfall_test.fc_day', 'rainfall_test.fc_hour'])

## 학습/테스트 데이터 시간/공간적 범위 분석

### 시간적 범위 분석

**년도(rainfall_ef_year)**

In [10]:
train_df["rainfall_train.ef_year"].unique()

array(['A', 'B', 'C'], dtype=object)

In [11]:
test_df["rainfall_test.ef_year"].unique()

array(['D'], dtype=object)

### 공간적 범위 분석


**지점(rainfall_train.stn4contest)**

In [ ]:
train_df["rainfall_train.stn4contest"].unique()

array(['STN001', 'STN002', 'STN003', 'STN004', 'STN005', 'STN006',
       'STN007', 'STN008', 'STN009', 'STN010', 'STN011', 'STN012',
       'STN013', 'STN014', 'STN015', 'STN016', 'STN017', 'STN018',
       'STN019', 'STN020'], dtype=object)

In [7]:
test_df["rainfall_test.stn4contest"].unique()

array(['STN031', 'STN032', 'STN033', 'STN034', 'STN035'], dtype=object)

**각각의 확률분포에 따라서 강수량 수치 계산: 각 계급별 중간 값 * 확률변수**
- 'rainfall_train.v01' : 0.1 -0.2  (mm)
- 'rainfall_train.v02' : 0.2 -0.5  (mm)
- 'rainfall_train.v03' : 0.5 -1.0  (mm)
- 'rainfall_train.v04' : 1.0 -2.0  (mm)
- 'rainfall_train.v05' : 2.0 -5.0  (mm)
- 'rainfall_train.v06' : 5.0 -10.0 (mm)
- 'rainfall_train.v07' : 10.0-20.0 (mm)
- 'rainfall_train.v08' : 20.0-30.0 (mm)
- 'rainfall_train.v09' : 30.0      (mm)

각각의 계급별 중간 값에 따라